In [1]:
import json
import numpy as np
import pandas as pd
from collections import Counter

from nltk import word_tokenize, sent_tokenize

from tqdm.auto import tqdm
tqdm.pandas()

pd.set_option('display.max_colwidth', 200)

In [2]:
with open("../data/href2synopsis.json", encoding="utf-8") as f:
    href2synopsis = json.load(f)

    Takeaways:
        - replace \n or \r sequences with single \n
        - use language-specific tokenizer
        - when using max_count_token_ratio make sure do delete stopwords
            (unless you specifically wish to locate examples with too much stopwords)
        - 

# BOW stats
    # TODO: add actual bow stats

In [24]:
data = pd.DataFrame()

data["href"] = href2synopsis.keys()
data["synopsis"] = href2synopsis.values()

In [25]:
data = data[data["synopsis"] != ""]

In [26]:
import regex as re

data["synopsis"] = data["synopsis"].apply(
    lambda x: re.sub("\s*\n\s*", "\n", x).strip()
)

In [75]:
# data["min_ord"] = data["synopsis"].progress_apply(
#     lambda s: min([ord(char) for char in s])
# )

# data["max_ord"] = data["synopsis"].progress_apply(
#     lambda s: max([ord(char) for char in s])
# )

# data["mean_ord"] = data["synopsis"].progress_apply(
#     lambda s: np.mean([ord(char) for char in s])
# )

# data["median_ord"] = data["synopsis"].progress_apply(
#     lambda s: np.median([ord(char) for char in s])
# )

# data["mode_ord"] = data["synopsis"].progress_apply(
#     lambda s: Counter([ord(char) for char in s]).most_common()[0][0]
# )

In [6]:
# data["mode_count"] = data["synopsis"].progress_apply(
#     lambda s: Counter(s).most_common()[0][1]
# )

# data["mode_ratio"] = data["synopsis"].progress_apply(
#     lambda s: Counter(s).most_common()[0][1]/len(s)
# )

# data["mean_char_count"] = data["synopsis"].progress_apply(
#     lambda s: np.mean(list(Counter(s).values()))
# )

# data["mean_token_count"] = data["synopsis"].progress_apply(
#     lambda s: np.mean(list(Counter([w.lower() for w in word_tokenize(s)]).values()))
# )

data["max_count_token_ratio"] = data["synopsis"].progress_apply(
    lambda s: Counter([w.lower() for w in word_tokenize(s)]).most_common()[0][1]/len(word_tokenize(s))
)

  0%|          | 0/18374 [00:00<?, ?it/s]

In [16]:
from stopwordsiso import stopwords

In [31]:
("w" in stopwords("en")) * True

1

In [38]:
def max_count_token_ratio_no_stops(text, stopwords=stopwords("en")):
    words = word_tokenize(text)
    counter = Counter()
    for w in words:
        lowered = w.lower()
        if lowered not in stopwords:
            counter.update([lowered])
    if not counter:
        return 0.
    return counter.most_common()[0][1]/len(words)

# data["max_count_token_ratio_stops"] = data["synopsis"].progress_apply(max_count_token_ratio_stops)
data["max_count_token_ratio_no_stops"] = data["synopsis"].progress_apply(max_count_token_ratio_no_stops)

  0%|          | 0/18374 [00:00<?, ?it/s]

In [12]:
data["char_total"] = data["synopsis"].progress_apply(len)
data["token_total"] = data["synopsis"].progress_apply(
    lambda x: len(word_tokenize(x))
)
data["sent_total"] = data["synopsis"].progress_apply(
    lambda x: len([sent for line in x.split("\n") for sent in  sent_tokenize(line) if line.strip()])
)

  0%|          | 0/18374 [00:00<?, ?it/s]

  0%|          | 0/18374 [00:00<?, ?it/s]

  0%|          | 0/18374 [00:00<?, ?it/s]

In [27]:
# data["lines_total"] = data["synopsis"].progress_apply(lambda x: len(x.split("\n")))

  0%|          | 0/19393 [00:00<?, ?it/s]

In [5]:
data["token_max_char_len"] = data["synopsis"].progress_apply(
    lambda x: max([len(t) for t in word_tokenize(x)]) if x.strip() else 0
)

data["sent_max_char_len"] = data["synopsis"].progress_apply(
    lambda x: max([len(t) for t in sent_tokenize(x)]) if x.strip() else 0
)

data["line_max_char_len"] = data["synopsis"].progress_apply(
    lambda x: max([len(t) for t in x.split("\n")]) if x.strip() else 0
)

  0%|          | 0/19393 [00:00<?, ?it/s]

  0%|          | 0/19393 [00:00<?, ?it/s]

  0%|          | 0/19393 [00:00<?, ?it/s]

In [16]:
data["sent_char_ratio"] = [t.sent_total/t.char_total for t in data.itertuples()]
data["sent_token_ratio"] = [t.sent_total/t.token_total for t in data.itertuples()]
data["token_char_ratio"] = [t.token_total/t.char_total for t in data.itertuples()]
data["token_sent_ratio"] = [t.token_total/t.sent_total for t in data.itertuples()]
data["char_sent_ratio"] = [t.char_total/t.sent_total for t in data.itertuples()]
data["char_token_ratio"] = [t.char_total/t.token_total for t in data.itertuples()]

In [20]:
data.describe().astype("float16")

,max_count_token_ratio
count,18368.000000
mean,0.072083
std,0.037292
min,0.026321
25%,0.054932
50%,0.065796
75%,0.081055
max,1.000000


In [42]:
stats = data.drop(columns=["href", "synopsis"]) #.drop(columns=["line_max_char_len"])

# Outlier detection

[The svm.OneClassSVM is known to be sensitive to outliers and thus does not perform very well for outlier detection](https://scikit-learn.org/stable/modules/outlier_detection.html#outlier-detection)

In [43]:
from sklearn.neighbors import LocalOutlierFactor
from sklearn.ensemble import IsolationForest

In [44]:
clf = LocalOutlierFactor(n_neighbors=20)
preds = clf.fit_predict(stats)

data["pred"] = preds
data["negative_outlier_factor_"] = clf.negative_outlier_factor_

In [45]:
# data[data["pred"] == -1]["synopsis"].sample(10).to_list()

In [46]:
data[data["pred"] == -1].sort_values(by="negative_outlier_factor_"
                                    )[['href', 'synopsis', 'pred', 'negative_outlier_factor_']].head(50)

,href,synopsis,pred,negative_outlier_factor_
19016,/14143-goong-1.5,Cast Interviews.,-1,-6.762185e+08
17165,/25378-innate-differences,BL Movie.,-1,-6.762185e+08
18883,/689819-hidamari-no-basho-hatsukoi,Mobile drama.,-1,-6.762185e+08
15964,/13390-sweet-20,Mini beauty drama.,-1,-3.298084e+08
11438,/65247-nct-dream-a-mid-summer-day-s-dream,NCT DREAM A Mid Summer Day's Dream,-1,-3.298084e+08
14405,/691835-mr-queen-the-story,Mr. Queen: Commentary,-1,-3.298084e+08
18052,/31000-replaylist,"Three different web dramas :\nFirst episode -Seventeen - Yang Yoseop - On the Road\nSecond episode - Not All Right, But It's Alright - Rothy - Butterfly Effect\nThird episode - Just One Bite - BEN...",-1,-2.462236e+08
7695,/734629-wicked-wealthy-women,A family. An inheritance. An accident. Nothing is an accident.\n(Source: DouQi7s Twitter),-1,-2.051308e+08
2276,/60733-kyou-kara-ore-wa-sp,Drama special to Kyou Kara Ore wa!!,-1,-1.792118e+08
11232,/713273-enhyphen-comeback-show-dimension-dilemma,Enhypen’s 1st full album’s comeback show!,-1,-1.553256e+08


In [35]:
data.assign(emperor_ratio=data["synopsis"].progress_apply(
    lambda s: len(re.findall("emperor", s.lower()))/len(word_tokenize(s))
)).sort_values(by="emperor_ratio", ascending=False).head(10)["synopsis"].to_list()

  0%|          | 0/18374 [00:00<?, ?it/s]

['It revolves around three generations of Emperor and their harem.',
 "Emperor Zhu Zhengde flees the palace after his mother picks an unappealing bride for him to marry. The emperor disguises himself as a commoner and attempts to live his life as one. Unfortunately, the emperor has always lived a pampered life and the emperor soon loses his money and his memory. The owner of an inn takes pity on this man and takes him in. Meanwhile, the emperor's evil uncle scours the land looking for the emperor and planning to take his throne.",
 'A tale of forbidden love between Qing Dynasty Emperor Kang Xi and Ming Dynasty Princess Huai Yu.',
 'Set in the final years of Emperor Kang Xi’s reign, The Palace will focus on Yin Xiang, the thirteenth son of Emperor Kang Xi, and the political upheavals and love stories of his youth.',
 'This movie is about the love story between Emperor Ming of Tang and the beautiful consort Yang Gui Fei.',
 "At the birthday banquet, the emperor asked to travel the world.